In [ ]:
import boto3
import os
import cv2
import io
from botocore.exceptions import ClientError

In [ ]:
minio_client = boto3.client(
    "s3",
    endpoint_url="http://localhost:9000",
    aws_access_key_id="admin",
    aws_secret_access_key="admin123"
)

In [ ]:
def upload_img(image_path, bucket_name="imgs"):
    try:
        minio_client.head_bucket(Bucket=bucket_name)
    except ClientError:
        minio_client.create_bucket(Bucket=bucket_name)

    object_name = image_path.replace("imgs/", "", 1)

    _, extension = os.path.splitext(object_name)
    extension = extension.lstrip(".")

    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Imagem {image_path} não encontrada!")

    _, buffer = cv2.imencode(f".{extension}", image)
    image_bytes = io.BytesIO(buffer)

    minio_client.put_object(
        Bucket=bucket_name,
        Key=object_name,
        Body=image_bytes.getvalue(),
        ContentType=f"image/{extension}"
    )
    print(f"Imagem {object_name} enviada com sucesso para o bucket '{bucket_name}'!")

def upload_imgs(images_path, bucket_name="imgs"):
    for image_path in images_path:
        upload_img(image_path, bucket_name)

In [ ]:
paths = ["imgs/animals/cat.jpeg", "imgs/animals/dog.jpg"]
upload_imgs(paths)